In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Data Collection and Analysis

Dataset: PIMA Diabetes dataset

In [2]:
# Loading the diabetes dataset to a pandas dataframe
diabetes_df = pd.read_csv('diabetes.csv')
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
diabetes_df.shape

(768, 9)

In [4]:
# Getting the statistical measures of the data
diabetes_df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
diabetes_df['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

    Non diabetic: 500 people
    Diabetic: 268 people

In [6]:
diabetes_df.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


People with diabetes are having higher Glucose level and higher in age than non diabetic people.

## Separate Features (X) and Target Variable (y)

In [7]:
X = diabetes_df.drop('Outcome', axis=1)
X.head(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32


In [8]:
y = diabetes_df['Outcome']
y.head(3)

0    1
1    0
2    1
Name: Outcome, dtype: int64

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=2)

"stratify=y" help ensure that both the training and testing sets have a similar distribution of the target y.

## Standardize the features

In [10]:
scaler = StandardScaler()

In [11]:
X_train = scaler.fit_transform(X_train)
X_train

array([[-1.13796489, -0.07971099, -3.5556072 , ...,  0.02825037,
        -0.98159708, -0.7885233 ],
       [ 0.64067858, -0.52091877,  0.02549599, ..., -0.17184452,
        -1.03823795,  0.31879426],
       [-0.84152431,  2.12632792, -0.48609018, ..., -0.25938604,
        -0.21545477,  2.19271628],
       ...,
       [ 2.12288146, -1.15121561,  0.23013046, ..., -0.25938604,
        -0.50760242,  0.14843771],
       [ 0.04779742, -0.30031488,  0.43476492, ...,  0.90366551,
        -0.69839272,  0.40397253],
       [-1.13796489, -1.11970076, -0.07682125, ...,  0.45345201,
        -0.69243053, -0.70334503]])

In [12]:
X_test = scaler.transform(X_test)
X_test

array([[-0.24864315, -0.48940393,  0.12781322, ..., -0.79714106,
        -0.78484458, -0.53298848],
       [-0.24864315, -0.26880004, -1.30462805, ..., -1.2223427 ,
        -0.98457818, -0.95887985],
       [ 0.344238  ,  0.80270457,  0.43476492, ...,  0.19082747,
        -0.7520525 ,  2.70378593],
       ...,
       [-0.54508373, -0.36334457,  0.23013046, ...,  0.02825037,
         0.67887478, -0.53298848],
       [-0.54508373, -1.27727497,  0.12781322, ..., -0.25938604,
         0.22872891, -0.70334503],
       [ 2.71576261, -0.48940393,  0.12781322, ...,  0.55349946,
        -0.87129644,  1.00022045]])

The data has undergone a standardization process to bring it into a similar range

In [13]:
X.shape, X_train.shape, X_test.shape

((768, 8), (614, 8), (154, 8))

# Training and Evaluating the model

## Model Training

In [15]:
# Define models with hyperparameter grids
models = {
    'Logistic Regression': (LogisticRegression(random_state=2), {
        'C': [0.001, 0.01, 0.1, 1, 10, 100]
    }),
    'Random Forest': (RandomForestClassifier(random_state=2), {
      'n_estimators': [50, 100, 200, 300],
      'max_depth': [None, 10, 20, 30]
    }),
    'SVM': (SVC(random_state=2), {
      'C': [0.1, 1, 10],
     'kernel': ['linear', 'rbf']
    }),
    'KNN': (KNeighborsClassifier(), {
      'n_neighbors': [3, 5, 7, 9],
     'p': [1, 2]
    }),
    'Decision Tree': (DecisionTreeClassifier(random_state=2), {
      'max_depth': [None, 10, 20, 30],
     'min_samples_split': [2, 5, 10]
    }),
    'Gradient Boosting': (GradientBoostingClassifier(random_state=2), {
      'n_estimators': [50, 100, 150],
     'learning_rate': [0.01, 0.1, 0.2]
    }),
    'Naive Bayes': (GaussianNB(), {}),
}

In [16]:
# Train and tune hyperparameters for each model
for name, (model, param_grid) in models.items():
    # Use GridSearchCV for hyperparameter tuning
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_
    
    # Evaluate the model on the training set
    y_train_pred = best_model.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)

    # Evaluate the model on the test set
    y_test_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    class_report = classification_report(y_test, y_test_pred)

    print(f"\n{name} Model (Tuned):")
    print(f"Best Hyperparameters: {grid_search.best_params_}")
    print(f"Training Set Accuracy: {train_accuracy:.2f}")
    print(f"Test Set Accuracy: {test_accuracy:.2f}")
    print("Classification Report:")
    print(class_report)

    # Cross-validation
    cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='accuracy')
    print(f"Cross-Validation Scores: {cv_scores}")
    print(f"Mean CV Accuracy: {cv_scores.mean():.2f}\n")


Logistic Regression Model (Tuned):
Best Hyperparameters: {'C': 1}
Training Set Accuracy: 0.79
Test Set Accuracy: 0.76
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.89      0.83       100
           1       0.72      0.52      0.60        54

    accuracy                           0.76       154
   macro avg       0.75      0.70      0.72       154
weighted avg       0.75      0.76      0.75       154

Cross-Validation Scores: [0.77272727 0.74675325 0.75974026 0.81699346 0.75163399]
Mean CV Accuracy: 0.77



C:\Users\UMER\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\UMER\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 


Random Forest Model (Tuned):
Best Hyperparameters: {'max_depth': 10, 'n_estimators': 50}
Training Set Accuracy: 1.00
Test Set Accuracy: 0.75
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.86      0.82       100
           1       0.67      0.54      0.60        54

    accuracy                           0.75       154
   macro avg       0.72      0.70      0.71       154
weighted avg       0.74      0.75      0.74       154

Cross-Validation Scores: [0.75324675 0.75324675 0.78571429 0.83006536 0.75816993]
Mean CV Accuracy: 0.78


SVM Model (Tuned):
Best Hyperparameters: {'C': 1, 'kernel': 'linear'}
Training Set Accuracy: 0.79
Test Set Accuracy: 0.77
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.91      0.84       100
           1       0.76      0.52      0.62        54

    accuracy                           0.77       154
   macro avg       0.77      0.71     